# Assignment

1 LDA: Latent Dirichlet allocation
2 With big data 1000 samples 
3 g ensim and sklearn to compare
4 n mf: non negative matrix factorization (stretch) 


In [28]:
#!pip install nltk
#!pip install spacy
#!pip install gensim
#!pip install pyLDAvis

     |████████████████████████████████| 1.6 MB 20.5 MB/s eta 0:00:01
     |████████████████████████████████| 548 kB 85.3 MB/s eta 0:00:01
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=96445 sha256=9e9de30987aaea3f8735585595f2212bcbf5375da5bb94f1db272eb684c69855
  Stored in directory: /home/ec2-user/.cache/pip/wheels/57/de/11/0a038be70c2c212ce45fa0f4f9da165bb5dd87de1288394dc3
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32056 sha256=9825d52a732d1f51acd73f246a8c69fa05d0b51748e97bb7bfd748ec191441e7
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5e/e0/ba/0dd7bb3f79264f8f60690da62918081bb8c3fb7442c38bfddd
Successfully built pyLDAvis funcy


In [29]:
import pandas as pd
import numpy as np
import os
import sklearn
import sys
np.set_printoptions(threshold=sys.maxsize)
from sklearn.model_selection import train_test_split
import re
from pprint import pprint
from glob import glob
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
path = "./sample_50/"

In [16]:
def get_abstract(soup):
    if soup.find("abstract") is None:
        return "NaN"
    
    return soup.find('abstract').text

In [3]:
from pathlib import Path
import glob
from bs4 import BeautifulSoup

In [9]:
def get_sample_data(path):
    pathlist = Path(path).glob('**/*.xml')
    abstracts = []
    for path in pathlist:
        # because path is object not string
        path_in_str = str(path)
        soup = BeautifulSoup(open('./'+ path_in_str, 'r'))
        abstracts.append(get_abstract(soup))
        # todo: get full articles?

    return np.array(abstracts)


In [17]:
X = get_sample_data(path)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/ec2-user/anaconda3/envs/python3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


# Assignment1: LDA: Latent Dirichlet allocation

## Using Sklearn

### Abstract as feature

Tokenize the words

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df = 0.95, min_df = 2, stop_words = 'english')
df = cv.fit_transform(X)

Apply LDA

In [19]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 7, random_state = 42)
lda.fit(df)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=7, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [50]:
for index, topic in enumerate(lda.components_):
    print(f'Top 15 words for Topic #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

Top 15 words for Topic #0
['results', 'rates', 'age', 'cases', 'data', 'based', 'higher', 'ci', 'services', 'likely', 'time', '95', 'screening', 'hiv', 'women']


Top 15 words for Topic #1
['disease', 'functioning', 'technologies', 'using', 'cancer', 'impact', 'behavior', 'adherence', 'electronic', 'model', 'use', 'smoking', 'data', 'information', 'health']


Top 15 words for Topic #2
['methods', 'study', 'identified', 'data', 'health', 'clinically', 'based', 'sensitivity', 'used', 'patient', 'relevant', 'populations', 'text', 'patients', 'sex']


Top 15 words for Topic #3
['profit', 'hospitals', 'non', 'public', 'health', 'evaluation', 'ethnic', 'parents', 'likely', 'mothers', 'process', 'using', 'testing', 'ethnically', 'response']


Top 15 words for Topic #4
['safety', 'medication', 'population', 'public', 'results', '95', 'multivariate', 'management', 'women', 'analysis', 'births', 'healthcare', 'exposure', 'pregnancy', 'patients']


Top 15 words for Topic #5
['participants', 'stud

### Paper contnet as feature TODO

## Using Gensim

### Abstract as feature 

In [30]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# data cleaning 

# Remove Emails
# data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# # Remove new line characters
# data = [re.sub('\s+', ' ', sent) for sent in data]

# # Remove distracting single quotes
# data = [re.sub("\'", "", sent) for sent in data]

Tokenization: tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.     

In [36]:
X.shape

(50,)

In [49]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(X))
data_words

[['community',
  'acquired',
  'lower',
  'respiratory',
  'tract',
  'infections',
  'lrti',
  'and',
  'pneumonia',
  'cap',
  'are',
  'common',
  'causes',
  'of',
  'morbidity',
  'and',
  'mortality',
  'among',
  'those',
  'aged',
  'years',
  'growing',
  'population',
  'in',
  'many',
  'countries',
  'detailed',
  'incidence',
  'estimates',
  'for',
  'these',
  'infections',
  'among',
  'older',
  'adults',
  'in',
  'the',
  'united',
  'kingdom',
  'uk',
  'are',
  'lacking',
  'we',
  'used',
  'electronic',
  'general',
  'practice',
  'records',
  'from',
  'the',
  'clinical',
  'practice',
  'research',
  'data',
  'link',
  'linked',
  'to',
  'hospital',
  'episode',
  'statistics',
  'inpatient',
  'data',
  'to',
  'estimate',
  'incidence',
  'of',
  'community',
  'acquired',
  'lrti',
  'and',
  'cap',
  'among',
  'uk',
  'older',
  'adults',
  'between',
  'april',
  'march',
  'by',
  'age',
  'sex',
  'region',
  'and',
  'deprivation',
  'quintile',
  

In [51]:
data_words[:1]

[['community',
  'acquired',
  'lower',
  'respiratory',
  'tract',
  'infections',
  'lrti',
  'and',
  'pneumonia',
  'cap',
  'are',
  'common',
  'causes',
  'of',
  'morbidity',
  'and',
  'mortality',
  'among',
  'those',
  'aged',
  'years',
  'growing',
  'population',
  'in',
  'many',
  'countries',
  'detailed',
  'incidence',
  'estimates',
  'for',
  'these',
  'infections',
  'among',
  'older',
  'adults',
  'in',
  'the',
  'united',
  'kingdom',
  'uk',
  'are',
  'lacking',
  'we',
  'used',
  'electronic',
  'general',
  'practice',
  'records',
  'from',
  'the',
  'clinical',
  'practice',
  'research',
  'data',
  'link',
  'linked',
  'to',
  'hospital',
  'episode',
  'statistics',
  'inpatient',
  'data',
  'to',
  'estimate',
  'incidence',
  'of',
  'community',
  'acquired',
  'lrti',
  'and',
  'cap',
  'among',
  'uk',
  'older',
  'adults',
  'between',
  'april',
  'march',
  'by',
  'age',
  'sex',
  'region',
  'and',
  'deprivation',
  'quintile',
  

Creating Bigram and Trigram Models

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.

In [52]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold, fewer phrases to form
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['community', 'acquired', 'lower', 'respiratory', 'tract', 'infections', 'lrti', 'and', 'pneumonia', 'cap', 'are', 'common', 'causes', 'of', 'morbidity', 'and', 'mortality', 'among', 'those', 'aged', 'years', 'growing', 'population', 'in', 'many', 'countries', 'detailed', 'incidence', 'estimates', 'for', 'these', 'infections', 'among', 'older', 'adults', 'in', 'the', 'united', 'kingdom', 'uk', 'are', 'lacking', 'we', 'used', 'electronic', 'general', 'practice', 'records', 'from', 'the', 'clinical', 'practice', 'research', 'data', 'link', 'linked', 'to', 'hospital', 'episode', 'statistics', 'inpatient', 'data', 'to', 'estimate', 'incidence', 'of', 'community', 'acquired', 'lrti', 'and', 'cap', 'among', 'uk', 'older', 'adults', 'between', 'april', 'march', 'by', 'age', 'sex', 'region', 'and', 'deprivation', 'quintile', 'levels', 'of', 'antibiotic', 'prescribing', 'were', 'also', 'assessed', 'lrti', 'incidence', 'increased', 'with', 'fluctuations', 'over', 'time', 'was', 'higher', 'in', '

Remove Stopwords, Make Bigrams and Lemmatize

The bigrams model is ready. Let’s define the functions to remove the stopwords, make bigrams and lemmatization and call them sequentially.

In [53]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [59]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# !python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/en_core_web_sm
-->
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [60]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['community', 'acquire', 'low', 'respiratory', 'tract', 'infection', 'common', 'cause', 'morbidity', 'mortality', 'aged', 'year', 'grow', 'population', 'many', 'country', 'detail', 'incidence', 'estimate', 'infection', 'old', 'adult', 'lack', 'use', 'electronic', 'general', 'practice', 'record', 'clinical', 'practice', 'research', 'datum', 'link', 'link', 'estimate', 'incidence', 'community', 'acquire', 'old', 'adult', 'sex', 'region', 'deprivation', 'quintile', 'level', 'antibiotic', 'prescribing', 'also', 'assess', 'incidence', 'increase', 'fluctuation', 'time', 'high', 'man', 'woman', 'age', 'increase', 'age', 'episode', 'person', 'year', 'year', 'year', 'cap', 'incidence', 'increase', 'markedly', 'age', 'episode', 'person', 'year', 'respectively', 'high', 'man', 'infection', 'group', 'increase', 'time', 'attenuate', 'age', 'standardisation', 'indicating', 'rise', 'largely', 'due', 'population', 'age', 'rate', 'deprive', 'quintile', 'higher', 'least', 'deprive', 'generally', 'high'

Create the Dictionary and Corpus needed for Topic Modeling

two main inputs to the LDA topic model are the dictionary(id2word) and the corpus.

eg: for the single doc, produce: a mapping of (word_id, word_frequency), (0,1) 1st word in the doc appear once

In [61]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 3), (2, 2), (3, 7), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 3), (10, 1), (11, 1), (12, 1), (13, 1), (14, 3), (15, 1), (16, 2), (17, 1), (18, 1), (19, 2), (20, 2), (21, 2), (22, 1), (23, 2), (24, 3), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 3), (31, 1), (32, 1), (33, 4), (34, 1), (35, 1), (36, 1), (37, 1), (38, 6), (39, 4), (40, 1), (41, 2), (42, 3), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 2), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 4), (60, 1), (61, 3), (62, 1), (63, 3), (64, 2), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 3), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 3), (87, 1), (88, 1), (89, 1), (90, 1), (91, 5)]]


#### id-word mapping:

In [64]:
id2word[0]

'account'

#### Item-frequency

In [63]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('account', 1),
  ('acquire', 3),
  ('adult', 2),
  ('age', 7),
  ('aged', 1),
  ('also', 1),
  ('antibiotic', 2),
  ('assess', 1),
  ('attenuate', 1),
  ('cap', 3),
  ('cause', 1),
  ('clinical', 1),
  ('common', 1),
  ('commonly', 1),
  ('community', 3),
  ('considerably', 1),
  ('country', 2),
  ('datum', 1),
  ('deprivation', 1),
  ('deprive', 2),
  ('detail', 2),
  ('due', 2),
  ('electronic', 1),
  ('episode', 2),
  ('estimate', 3),
  ('finding', 1),
  ('first', 1),
  ('fluctuation', 1),
  ('general', 1),
  ('generally', 1),
  ('group', 3),
  ('grow', 1),
  ('health', 1),
  ('high', 4),
  ('higher', 1),
  ('highly', 1),
  ('hospitalisation', 1),
  ('immediate', 1),
  ('incidence', 6),
  ('increase', 4),
  ('indicating', 1),
  ('individual', 2),
  ('infection', 3),
  ('lack', 1),
  ('largely', 1),
  ('least', 1),
  ('level', 1),
  ('likely', 1),
  ('link', 2),
  ('long', 1),
  ('low', 2),
  ('lrti', 1),
  ('man', 2),
  ('many', 1),
  ('markedly', 1),
  ('methodology', 1),
  ('mo

Building the Topic Model

In [65]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

View the topics in LDA model

In [67]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.031*"model" + 0.027*"frequent" + 0.023*"statistic" + 0.020*"persistent" + '
  '0.020*"network" + 0.016*"use" + 0.016*"validation" + 0.016*"attender" + '
  '0.012*"patient" + 0.012*"calibration"'),
 (1,
  '0.068*"alarm" + 0.030*"relevant" + 0.026*"patient" + 0.026*"clinically" + '
  '0.017*"clinical" + 0.017*"proportion" + 0.013*"direct" + 0.013*"technical" '
  '+ 0.013*"reduce" + 0.013*"pressure"'),
 (2,
  '0.017*"patient" + 0.016*"use" + 0.015*"risk" + 0.015*"readmission" + '
  '0.015*"case" + 0.015*"system" + 0.013*"result" + 0.012*"intake" + '
  '0.010*"social" + 0.010*"category"'),
 (3,
  '0.020*"code" + 0.018*"text" + 0.018*"sex" + 0.018*"clonal" + '
  '0.018*"population" + 0.016*"keyword" + 0.016*"adaptation" + 0.014*"patient" '
  '+ 0.011*"free" + 0.011*"interference"'),
 (4,
  '0.038*"birth" + 0.035*"indigenous" + 0.014*"outcome" + 0.014*"cohort" + '
  '0.014*"baby" + 0.014*"non" + 0.011*"likely" + 0.011*"mother" + 0.011*"time" '
  '+ 0.011*"low"'),
 (5,
  '0.001*"rep

Compute Model Perplexity and Coherence Score

Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is.

In [68]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.044682366521396

Coherence Score:  0.49742307895379645


Visualize the topics-keywords

In [69]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.122448  0.023997       1        1  10.610724
11    -0.116047 -0.108915       2        1   9.508956
2     -0.027255  0.033330       3        1   8.532743
10    -0.067574  0.075345       4        1   8.423749
19    -0.101325 -0.125333       5        1   7.935310
8     -0.099998  0.043990       6        1   7.283770
14    -0.004891  0.110835       7        1   6.036489
15    -0.093033 -0.056152       8        1   5.890178
3      0.050590  0.129830       9        1   5.855007
17    -0.009305  0.064213      10        1   5.631329
12    -0.023055  0.045976      11        1   4.309558
18     0.150898 -0.066872      12        1   3.527978
4      0.017348 -0.062996      13        1   3.331254
0      0.054149  0.069561      14        1   2.820471
16     0.045306 -0.028794      15        1   2.583911
7      0.034328  0.006512      16        1   2.426502
1      0.061936  0.011021      17        1   2.410662
13     0.079872 -0.101470      18        1   2.049254
9      0.101775 -0.038429      19        1   0.815125
5      0.068730 -0.025650      20        1   0.017034, topic_info=             Term       Freq      Total Category  logprob  loglift
1576        alarm  10.000000  10.000000  Default  30.0000  30.0000
232        system  34.000000  34.000000  Default  29.0000  29.0000
264   information  32.000000  32.000000  Default  28.0000  28.0000
208       patient  60.000000  60.000000  Default  27.0000  27.0000
235          text  17.000000  17.000000  Default  26.0000  26.0000
...           ...        ...        ...      ...      ...      ...
222        result   0.000689  40.795761  Topic20  -7.3150  -2.3107
208       patient   0.000686  60.981148  Topic20  -7.3205  -2.7181
368       include   0.000678  20.344812  Topic20  -7.3323  -1.6322
412      identify   0.000670  21.581161  Topic20  -7.3436  -1.7025
351          base   0.000670  24.081873  Topic20  -7.3441  -1.8127

[1080 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1673      2  0.793767   abuse
878       6  0.838296  accept
384       1  0.152837  access
384       2  0.076418  access
384       4  0.152837  access
...     ...       ...     ...
91       13  0.067955    year
91       16  0.135911    year
91       17  0.067955    year
613       1  0.450542   young
613      16  0.450542   young

[1868 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 12, 3, 11, 20, 9, 15, 16, 4, 18, 13, 19, 5, 1, 17, 8, 2, 14, 10, 6])

Finding the dominant topic in each sentence

Find the most representative document for each topic

Topic distribution across documents

# TODO

- Building LDA Mallet Model
- at least 1000 data sample instead of 50

# Reference

[1]:https://www.youtube.com/watch?v=T05t-SqKArY

[1] LDA: https://www.youtube.com/watch?v=T05t-SqKArY

[2] https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

[3] https://medium.com/analytics-vidhya/topic-modelling-using-latent-dirichlet-allocation-in-scikit-learn-7daf770406c4